In [1]:
import pandas as pd
import re
sample = ['http://1.1.1.1',
          'https://1.2.3.4',
          'http://10.1.2.3',
          None,
          'https://www.google.com']
# feel free to use a more precise regex if you want.
re_IPv4_url = re.compile('https?:\/\/([0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3})')
df = pd.DataFrame(sample,columns=['referrer'])

Make a DataFrame with some mixed data:

In [2]:
df

,referrer
0,http://1.1.1.1
1,https://1.2.3.4
2,http://10.1.2.3
3,None
4,https://www.google.com


Extract referrers that contain IPv4 literals:

In [3]:
literal_ip_referrers = df[df.referrer.fillna('').str.contains(re_IPv4_url)].referrer.unique()
literal_ip_referrers

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


array(['http://1.1.1.1', 'https://1.2.3.4', 'http://10.1.2.3'],
      dtype=object)

Extract just the IP addresses:

In [4]:
ip_addrs = df.referrer.fillna('').str.extract(re_IPv4_url).dropna()
ip_addrs

,0
0,1.1.1.1
1,1.2.3.4
2,10.1.2.3


Rename the column for readability:

In [5]:
ip_addrs.columns = ['ip']

In [6]:
ip_addrs

,ip
0,1.1.1.1
1,1.2.3.4
2,10.1.2.3


In [7]:
ip_addrs

,ip
0,1.1.1.1
1,1.2.3.4
2,10.1.2.3


Convert the addresses into IPv4Network objects:

In [8]:
from ipaddress import ip_address,ip_network
ip_addrs.ip = ip_addrs.ip.apply(ip_address)

In [9]:
ip_addrs

,ip
0,1.1.1.1
1,1.2.3.4
2,10.1.2.3


In [10]:
type(ip_addrs)

pandas.core.frame.DataFrame

In [11]:
type(ip_addrs.ip.loc[0])

ipaddress.IPv4Address

Filter them for membership in a particular network:

In [12]:
ip_addrs[ip_addrs.ip.apply(lambda x: x in ip_network('1.2.3.0/24'))]

,ip
1,1.2.3.4
